In [ ]:
import json
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
{"tags": ["hide-input",]}

mainfile = "result_2023_6_8.json"
maindata = []
with open(mainfile, 'r') as f:
  for entry in f:
      oneEntry = json.loads(entry)
      maindata.append(oneEntry)

# Data Analysis for JSON Files (draft)
Looking into website leak data from edutech extension. Written up version of exploratory data analysis. <br>
**Mentor:** Jake Chanesson,   **Research Assistant:** Shayona Basu

Data produced by **AirLAB**

### Removing data entries where the json file has no report data saved:

In [ ]:
filtered_data = []
for d in maindata: 
  if d.get('report') != None: #not empty
    if d.get('report').get('user_id') != None: 
      filtered_data.append(d)

In [ ]:
f"After the initial filtiration there are {len(filtered_data)} entries, when there were initially {len(maindata)} entries"

I am curious to check what actually consists of empty data. I am defining empty data as data where reports are made, but there is no user_id. I am going to go through and explore what the empty data consists of. I believe this can be useful to understand we are not using, and confirming it is empty. 

In [ ]:
empty_data = []
for d in maindata: 
    if d.get('report') != None: 
        if d.get('report').get('user_id') == None:
            empty_data.append(d)
print(f"There are {len(empty_data)} entries of json data with reports but no user")

Printing a random entry of the empty dataset

In [ ]:
empty_data[38473]

From checking this random entry, we see there is seemingly useful amount of information, so I think we should edit how our first stage of filtering, and potentially include reports where there is no user_id. We can save the data with no user_id, in its own group and continue to do website analysis

In [ ]:
print(f'Of all the data entries from the files, the initial filtration removes {round(len(empty_data)/len(maindata) *100,2)}%' )

In [ ]:
filtered_data1 = []
for d in maindata: 
  if d.get('report') != None: #not empty
    if d.get('report').get('initiator_domain') != None: 
        filtered_data1.append(d)

I think maybe a better initial filtering would be to remove data where the initiator domain data is not collected. I think now is a good time to define two important cateogires for websites collected, throughout the rest of this data analysis <br>
**Initiator Domain:**   The website where the user lands, where the leak then occurs. 
**Tracker URL:**   The website of which the tracker owns, and where the leak actually happens

Documentation: https://homes.esat.kuleuven.be/~asenol/leaky-forms/ 

In [ ]:
print(f'There are {len(filtered_data1)} websites that have the leak origin website data. Accordingly, there were {len(maindata)-len(filtered_data1)} entries, or {round(((len(maindata)-len(filtered_data1))/len(maindata))*100,2)}% removed in this updated initial filtration.')


### Here is an example entry of the new filtered dataset: 

In [ ]:
filtered_data1[200000]

Now, I will begin to make sense of reduced dataset, `filtered_data1`. As per the definitions a few cells above, I will now go through the filtered data set, and collect the Initiator URL and the Leak URL, and present it in a table.

In [ ]:
#Saving Leak_URLS and Origin_URLS
Leak_URLS = []
Origin_URLS = []
for d in filtered_data1: 
    rep = d['report']
    Origin_URLS.append(rep['initiator_domain'])
    if rep.get('tracker_info') != None: 
        #some reports do not have tracker info
        if rep['tracker_info'].get('tracker').get('owner').get('url') != None: 
            #all reports with tracker info, have the following
            Leak_URLS.append(d['report']['tracker_info']['tracker']['owner']['url'])
        else: Leak_URLS.append("None")
    else: Leak_URLS.append("None")

Since writing data into a Pandas Dataframe requires both columns to be of the same length, I have to ammend my code a bit, by adding the string "None" when there is not a Leak_URl recorded. Here I am just ensuring both lists, which will be inputted as future column data, are the same length:

In [ ]:
len(Leak_URLS), len(Origin_URLS)

In [ ]:
df_leak_origin_urls = pd.DataFrame.from_dict({'Leak URLS': Leak_URLS, 'Origin URLS': Origin_URLS})
df_leak_origin_urls.head(20)

This is interesting, but presenting the data in this way is not letting us see a clear picture of what is happening. I want to find a way to get more useful data from this. First, let me start with seeing more of the origin urls. 

In [ ]:
print(f'There are {len(set(Origin_URLS))} unique origin urls')

Writing a simple function that goes through a list and produces a dictionairy that counts how many times that entry appears, and sorts it in ascending order. I will then pass origin_urls through, and see what are the top websites where leaks occur from. The function returns it as a dictionairy, but I will present it as a table. 

In [ ]:
def simple_produce_count(list):
    count_d = {}
    for i in list: 
        if i in count_d: 
            a = count_d.get(i)
            count_d[i] = a + 1
        else: 
            count_d[i] = 1
            
    return dict(sorted(count_d.items(), key=lambda x:x[1], reverse = True))

In [ ]:
orig_urls = simple_produce_count(Origin_URLS)
df_leak_origin_urls = pd.DataFrame.from_dict({'Origin of Leak Website URLS': list(orig_urls.keys()), 'Appearing': list(orig_urls.values())})
df_leak_origin_urls['Proportion (%)'] = round((df_leak_origin_urls.Appearing/len(Leak_URLS))*100,2)
df_leak_origin_urls.rename_axis("Count").head(30)

*In the above table, I displayed the top 30 websites from the total 65 leak websites found in the data, for consideration of space. I will allow all tables to be presented in full format in the appendix*  <br>

#### Analysis on Google Docs being the source of the most leaks
So, we see that 38.2% of the leaks originate from Google Docs. It is also relevant to note that Google Docs is heavily used by students to write classwork and class projects. Furthur, the Google enviroment is a popular system that schools integrate into learning, and students store most of their school or personal related information. Therefore leaks relating to this would be of significance to educator's who care about student's online security. <br>

We see google.com, the search platform being responsible for 17.18% of the leaks, or 60,847 leaked data from students. Google Mail takes up 5.38% of the leaks, and Google Calender takes up 2.16%. The last significant source of leaks from the Google ecosystem is Google Calendar which has 2.16%.


### What leak websites collect the leak data from the origin websites?
A next step in this analysis could be to connect what leak websites collect data from these websites. It would be interesting to see if there is a relationship from websites where the leak is taking place, and what the tracker collects is of the leak website.  